# Prelude

In [1]:
from pysat.formula import CNF
from pysat.solvers import *
import re
import pandas as pd
import os

CONFIG_DIR = "../linux-4.14.152"
DIMACS_FLA = 'examples/out.dimacs'
CONFIG = "../linux-4.14.152/.config"
CONF_FLAG = ["randconfig", "defconfig", "allnoconfig", "allyesconfig", "allmodconfig", "alldefconfig"]
SOLU_SIZE = 1000 # Initial number of solutions to extract

# Loading data

In [2]:
#
#
variables = {}
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c"):
            m = re.search('(c) (\d+) (\w+)', line)
            var_id = int(m.group(2))
            var_name = m.group(3)  
            variables[var_id] = var_name
            
def search_kconfig_type(option_name, df_options):
    ktype = df_options.query("option == " + '"' + option_name + '"')['type']
    return ktype.values[0]

# we are thinking the DIMACS id (number k) corresponding to the "module" variable of an option
def search_kmodule(option_name):
    for k, v in variables.items():
        if v == option_name + "_MODULE":
            return k

# Build a dict {option: val} from .config file
def get_dict(filename):
    res = dict()
    with open(filename, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res

# Extracting solutions

In [3]:
# Extract the SOLU_SIZE solutions
fla = CNF(DIMACS_FLA)
models = []
with Solver(bootstrap_with=fla.clauses) as s:
    s.solve()
    for no, k in enumerate(s.enum_models(), 1):
        if no > SOLU_SIZE:
            break
        models.append(k)

In [4]:
# Erase every variables that are not "real" features
config_models = list(map(lambda x: {variables[k] for k in x if k > 0 and k in variables}, models))

In [5]:
# Get rid of redundant solutions
lon = len(config_models)
res = list()
current = 0
in_list = True
while(in_list):
    res.append(config_models[current])
    for k in range(current + 1, lon):
        if config_models[current] != config_models[k]:
            current = k
            break
        if k == lon - 1:
            in_list = False

In [6]:
potential_configurations = res
len(potential_configurations)

108

# Config Generation

In [7]:
# Dictionary {option : [possible_values+]}
feature = dict()
with open(DIMACS_FLA, 'r') as f:
    dimacs_lines = f.readlines()
    for line in dimacs_lines:
        if line.startswith("c") and '=' in line:
            m = re.search('(c) (\d+) (\w+)=([\w"-/]*)', line)
#             print(m.group(3), m.group(4))
            if m.group(4) != 'nonempty' and m.group(4) != 'n':
                if m.group(3) in feature:
                    feature[m.group(3)].append(m.group(4))
                else:
                    feature[m.group(3)] = [m.group(4)]

In [8]:
import random

# Write .config content using the solution and 
# pick a random value in the list as real value in
# the configuration
def create_file_content(res):
    df_options = pd.read_csv('examples/alloptions-x86.4.14.152.csv')
    # file content
    file = ""
    for elt in res:
        if "CHOICE_" in elt:
            continue
        kconfig_type = search_kconfig_type(elt, df_options)
        if kconfig_type == 'BOOL':
    #         print("CONFIG_{}=y".format(elt))
            file += "CONFIG_{}=y\n".format(elt)
        elif kconfig_type == 'TRISTATE':
            file += "CONFIG_{}=y\n".format(elt)
        elif kconfig_type == "STRING":
            if elt == "IMA_DEFAULT_HASH":
                found = False
                for k in res:
                    if "IMA_DEFAULT_HASH_" in k and k != "IMA_DEFAULT_HASH":
                        file += 'CONFIG_{}="{}"\n'.format(elt, k[17:].lower())
                        found = True
                if not found:
                    file += 'CONFIG_{}=""\n'.format(elt)
            else:
                file += 'CONFIG_{}="{}"\n'.format(elt, random.choice(feature[elt]))
        elif kconfig_type == "INT":
            file += "CONFIG_{}={}\n".format(elt, random.choice(feature[elt]))
        elif kconfig_type == "HEX":
            file += "CONFIG_{}={}\n".format(elt, random.choice(feature[elt]))
    return file

In [9]:
# Create .config (experimentation)
def create_config_xp(res):
    file_content = create_file_content(res)
    with open('../linux-4.14.152/.config', 'w') as stream:
        stream.write("# EXPERIMENTATION\n##################\n")
        stream.write(file_content)

In [10]:
# potential_configurations = potential_configurations[:3]


In [11]:
diff_opt_occ = dict()
for solu in potential_configurations:
    create_config_xp(solu)
    os.system("make olddefconfig -C {}".format(CONFIG_DIR))
    new_config = get_dict("{}/.config".format(CONFIG_DIR))
    # diff
    if len(new_config) > len(solu):
        diff = set(new_config) - solu
        for feat in diff:
            if feat in diff_opt_occ:
                diff_opt_occ[feat] += 1
            else:
                diff_opt_occ[feat] = 1
    

In [12]:
diff_opt_occ

{'USB_PCI': 108,
 'IR_MCE_KBD_DECODER': 108,
 'TRACEPOINTS': 108,
 'HW_RANDOM_MTK': 108,
 'FRAMEBUFFER_CONSOLE_DETECT_PRIMARY': 108,
 'WLAN_VENDOR_TI': 108,
 'SND_VERBOSE_PROCFS': 108,
 'HID_ITE': 108,
 'LDISC_AUTOLOAD': 108,
 'PREVENT_FIRMWARE_BUILD': 108,
 'AIC79XX_REG_PRETTY_PRINT': 108,
 'RT2800PCI_RT53XX': 108,
 'X86_MCE_THRESHOLD': 108,
 'DVB_NET': 108,
 'CAN_BCM': 108,
 'IR_XMP_DECODER': 108,
 'SCSI_PROC_FS': 108,
 'RSI_USB': 108,
 'HID_PICOLCD_LCD': 108,
 'SECTION_MISMATCH_WARN_ONLY': 108,
 'FB_RADEON_I2C': 108,
 'ACPI_SLEEP': 108,
 'HW_RANDOM': 108,
 'KVM_GUEST': 108,
 'VIDEO_GO7007_LOADER': 108,
 'COMPAT_BRK': 108,
 'HID_BELKIN': 108,
 'NET_VENDOR_VIA': 108,
 'RT2800USB_RT33XX': 108,
 'NET_VENDOR_SYNOPSYS': 108,
 'IDEPCI_PCIBUS_ORDER': 108,
 'SH_DMAE_BASE': 108,
 'LOGO_LINUX_CLUT224': 108,
 'UBIFS_FS_SECURITY': 108,
 'CRYPTO_LZO': 108,
 'CPU_IDLE': 108,
 'HID_KENSINGTON': 108,
 'HID_A4TECH': 108,
 'INET6_XFRM_MODE_TRANSPORT': 108,
 'CONTEXT_TRACKING_FORCE': 108,
 'HISI_THERMA

In [13]:
for k in diff_opt_occ.keys():
    if diff_opt_occ[k] != 108:
        print(k)

CAPI_TRACE
COMMON_CLK_XGENE
BOUNCE
XEN_SCRUB_PAGES
SQUASHFS_ZLIB
CAN_CALC_BITTIMING
MSDOS_PARTITION
SCSI_SAS_HOST_SMP
OCFS2_FS_STATS
FB_ATY_BACKLIGHT


In [27]:
len(diff_opt_occ)

414

In [15]:
# Saving data
file_content = "option;occurence\n"
for option in diff_opt_occ.keys():
    file_content += "{};{}\n".format(option, diff_opt_occ[option])
with open("examples/diff_occ.csv", "w") as stream:
    stream.write(file_content)

In [16]:
# Read file of impossible values
def read_imp_val_csv(file):
    df = pd.read_csv(file)
    df_dict = df.to_dict()
    res = dict()
    for k, v in df_dict['option'].items():
        imp = map(lambda x: x.replace("'", ''), 
                  df_dict['impossible_value'][k][1:-1].split()) 
        res[v] = set(imp)
    return res
impossible_values = read_imp_val_csv('examples/impossible.csv')

In [28]:
# count core features from diff dict
core_f = 0
hasnt_iv = 0
for opt in diff_opt_occ:
    if opt in impossible_values :
        if 'n' in impossible_values[opt]:
            core_f += 1
    else:
        hasnt_iv += 1
print("{}/{} have not impossible values\n{}/{} of core features".format(hasnt_iv, len(diff_opt_occ),
                                                                        core_f, len(diff_opt_occ)))

414/414 have not impossible values
0/414 of core features


In [23]:
has_iv

414

In [29]:
len(config_models[0])

2182